In [5]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import JSONLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
import os
import json

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")

In [6]:
# JSON 파일 로드
json_file = "programs.json"
with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)
print(data)

# 문서를 LangChain에서 사용할 형식으로 변환
documents = []
for program in data["프로그램_정보"]:
    text = json.dumps(program, ensure_ascii=False)
    documents.append(text)

# OpenAI 임베딩 모델 사용
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# Chroma 벡터스토어에 저장
vectorstore = Chroma.from_texts(documents, embeddings)

{'프로그램_정보': [{'제목': 'JJ 취업컨설팅 Day(2월)', '설명': '고학년 대상 취업 상담 및 컨설팅, 입사지원서 및 면접 컨설팅 실시. 담당 취업지원관과 1:1 상담 진행.', '신청기간': '2025.01.24 ~ 2025.02.21', '기간': '2025.02.03 ~ 2025.02.28', '장소': '스타센터 240호 (대학일자리플러스센터)', '혜택': '재맞고 점프업 포인트 1시간 인정', '신청대상': '3~4학년 재학생, 졸업유예자', '운영시간': '월~금 (공휴일 제외), 1인 최대 50분 (9:00~12:00, 13:30~16:30)', '문의처': '063-220-2799'}, {'제목': '동계방학 NCS 필기 마스터 교육', '설명': '공기업 취업을 희망하는 학생들을 위한 NCS 필기전형 마스터 교육. 공기업 채용동향 및 NCS과목별 풀이 팁 제공.', '신청기간': '2025.02.03 ~ 2025.02.11', '기간': '2025.02.13 ~ 2025.02.14', '장소': '가상공간 온라인 (실시간 강의실 100)', '혜택': '점프업 포인트 10점', '신청대상': '공기업 취업을 희망하는 재학생 및 졸업생', '운영시간': '2월 13일(목) 13:00~17:00 - 2025년 공기업 채용동향(2h), 수리영역 특강(2h) / 2월 14일(금) 13:00~17:00 - NCS 문제해결능력 특강(2h), 자원관리능력 특강(2h)', '추가사항': '신청 희망자는 첨부파일 내 확약서를 작성 후 담당자에게 메일 제출', '문의처': '063-220-4652 (대학일자리플러스센터 김은미, 이메일: fromem@jj.ac.kr)'}, {'제목': '2025학년도 선배학습멘토 멘토링 캠프', '설명': '전주대학교 Residential College 멘토링 캠프. 신입생 멘토링을 위한 선배 학습 멘토 교육.', '신청기간': '2025.02.07 ~ 2025.02.09', '기간': '2025.02.10 ~ 

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************d1AA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
import os
import requests
import streamlit as st
from bs4 import BeautifulSoup
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage

# 환경 변수 설정
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = ""
os.environ["LANGCHAIN_ENDPOINT"] = ""
os.environ["LANGCHAIN_PROJECT"] = ""

# LangChain OpenAI 모델 초기화
chat_model = ChatOpenAI(model_name="gpt-4o", temperature=0.7)

# Streamlit UI 구성
st.set_page_config(page_title="전주대학교 비교과 챗봇", page_icon="🎓", layout="centered")
st.write("✅ Streamlit이 정상적으로 실행되고 있습니다!")

st.title("🎓 전주대학교 비교과 챗봇")
st.write("전주대학교 비교과 프로그램에 대해 질문하세요!")

# ✅ **세션 상태 강제 초기화 (에러 방지)**
if "messages" not in st.session_state:
    st.session_state["messages"] = [
        SystemMessage(content="당신은 전주대학교 비교과 프로그램을 안내하는 챗봇입니다.")
    ]

# JSON 파일에서 비교과 프로그램 정보 로드
@st.cache_data
def load_program_data():
    file_path = r"C:\Users\user\Desktop\Github\prjRepo_JJU\project\programs.json"  # 업로드된 JSON 파일 경로
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
        return data.get("프로그램_정보", [])
    except Exception as e:
        return [{"제목": "데이터 로드 오류", "설명": str(e)}]

program_data = load_program_data()

# 비교과 프로그램 정보를 기반으로 검색
def find_program(query):
    results = []
    for program in program_data:
        if query.lower() in program["제목"].lower() or query.lower() in program["설명"].lower():
            results.append(program)
    return results

# ✅ **KeyError 방지: .get()을 사용하여 안전하게 데이터 접근**
messages = st.session_state.get("messages", [])

# 채팅 UI
chat_container = st.container()

# 대화 기록 표시
for msg in messages:
    with chat_container:
        if isinstance(msg, HumanMessage):
            with st.chat_message("user"):
                st.write(msg.content)
        elif isinstance(msg, AIMessage):
            with st.chat_message("assistant"):
                st.write(msg.content)

# 사용자 입력
user_input = st.chat_input("비교과 프로그램에 대해 질문하세요!")

if user_input:
    # 사용자 메시지 추가
    messages.append(HumanMessage(content=user_input))
    st.session_state["messages"] = messages  # 세션 상태 업데이트
    
    # ✅ 특정 월의 마감 프로그램을 묻는 경우 고정된 응답 반환
    if any(keyword in user_input for keyword in ["월", "마감", "신청 종료", "언제까지", "기한"]):
        response_content = (
            "📌 죄송하지만, 특정 월에 마감하는 비교과 프로그램에 대한 실시간 정보는 제공할 수 없습니다. "
            "전주대학교 비교과 프로그램에 대한 자세한 정보는 전주대학교 공식 웹사이트나 관련 부서에 문의하시기 바랍니다. "
            "일반적으로 학교의 공지사항, 학사 일정, 또는 해당 부서의 게시판을 통해 프로그램 마감일을 확인할 수 있습니다."
        )
    else:
        # 비교과 프로그램 검색
        program_results = find_program(user_input)

        if program_results:
            response_content = "\n\n".join(
                [f"**{p['제목']}**\n설명: {p['설명']}\n기간: {p['기간']}\n장소: {p['장소']}\n신청대상: {p['신청대상']}\n문의처: {p['문의처']}" for p in program_results]
            )
        else:
            response = chat_model(messages)
            response_content = response.content
    
    # AI 메시지 추가
    messages.append(AIMessage(content=response_content))
    st.session_state["messages"] = messages  # 세션 상태 업데이트
    
    # 채팅 UI 즉시 갱신
    with chat_container:
        with st.chat_message("user"):
            st.write(user_input)
        with st.chat_message("assistant"):
            st.write(response_content)


2025-02-10 10:06:16.038 No runtime found, using MemoryCacheStorageManager


In [9]:
import os
import json
import requests
import streamlit as st
from bs4 import BeautifulSoup
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage

# 환경 변수 설정
os.environ["OPENAI_API_KEY"] = "sk-..."  # 보안상 실제 키는 제거함
os.environ["LANGCHAIN_API_KEY"] = "lsv2_..."
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "chatbot-test2"

# LangChain OpenAI 모델 초기화
chat_model = ChatOpenAI(model_name="gpt-4o", temperature=0.7)

# Streamlit UI 구성
st.set_page_config(page_title="전주대학교 비교과 챗봇", page_icon="🎓", layout="centered")
st.write("✅ Streamlit이 정상적으로 실행되고 있습니다!")

st.title("🎓 전주대학교 비교과 챗봇")
st.write("전주대학교 비교과 프로그램에 대해 질문하세요!")

# JSON 파일에서 비교과 프로그램 정보 로드
@st.cache_data
def load_program_data():
    file_path = r"C:\Users\user\Desktop\Github\prjRepo_JJU\project\programs.json"  # 업로드된 JSON 파일 경로
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
        return data["프로그램_정보"]
    except Exception as e:
        return [{"제목": "데이터 로드 오류", "설명": str(e)}]

program_data = load_program_data()

# 비교과 프로그램 정보를 기반으로 검색
def find_program(query):
    results = []
    for program in program_data:
        if query in program["제목"] or query in program["설명"]:
            results.append(program)
    return results

# 대화 기록 저장
if "messages" not in st.session_state:
    st.session_state.messages = [
        SystemMessage(content="당신은 전주대학교 비교과 프로그램을 안내하는 챗봇입니다.")
    ]

# 채팅 UI
chat_container = st.container()

# 대화 기록 표시
for msg in st.session_state.messages:
    with chat_container:
        if isinstance(msg, HumanMessage):
            with st.chat_message("user"):
                st.write(msg.content)
        elif isinstance(msg, AIMessage):
            with st.chat_message("assistant"):
                st.write(msg.content)

# 사용자 입력
user_input = st.chat_input("비교과 프로그램에 대해 질문하세요!")

if user_input:
    # 사용자 메시지 추가
    st.session_state.messages.append(HumanMessage(content=user_input))
    
    # 비교과 프로그램 검색
    program_results = find_program(user_input)

    if program_results:
        response_content = "\n\n".join(
            [f"**{p['제목']}**\n설명: {p['설명']}\n기간: {p['기간']}\n장소: {p['장소']}\n신청대상: {p['신청대상']}\n문의처: {p['문의처']}" for p in program_results]
        )
    else:
        response = chat_model(st.session_state.messages)
        response_content = response.content
    
    # AI 메시지 추가
    st.session_state.messages.append(AIMessage(content=response_content))
    
    # 채팅 UI 즉시 갱신
    with chat_container:
        with st.chat_message("user"):
            st.write(user_input)
        with st.chat_message("assistant"):
            st.write(response_content)


2025-02-10 09:57:47.618 No runtime found, using MemoryCacheStorageManager
2025-02-10 09:57:47.620 No runtime found, using MemoryCacheStorageManager


AttributeError: st.session_state has no attribute "messages". Did you forget to initialize it? More info: https://docs.streamlit.io/library/advanced-features/session-state#initialization